In [6]:
from datasets import load_dataset
import tqdm as notebook_tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np


# Unet architecture

We will try to first build our model based on the Unet architecture because from our personal research, this architecture seems to be commonly used for image segmentation.

In [7]:
# Set random seed for reproducibility
torch.manual_seed(42)


image_train_dataset = load_dataset(
'keremberke/satellite-building-segmentation',
split='train', name='mini'
).to_pandas()
image_train_dataset.head()

,image_id,image,width,height,objects
0,1517,"{'bytes': None, 'path': 'C:\Users\princ\.cache...",500,500,"{'id': [20602, 20603, 20604, 20605, 20606], 'a..."
1,1030,"{'bytes': None, 'path': 'C:\Users\princ\.cache...",500,500,"{'id': [14034, 14035, 14036, 14037, 14038, 140..."
2,1217,"{'bytes': None, 'path': 'C:\Users\princ\.cache...",500,500,"{'id': [16678, 16679, 16680, 16681, 16682, 166..."


In [9]:
# Classes needed for the model definition

class layer_conv2():
    # Give as parameter the input and output channel
    def __init__(self, input_channel, output_channel):
        super(layer_conv2, self).__init__()
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channel),
            nn.ReLU(),
            nn.Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=3, padding=1),
            nn.BatchNorm2d(output_channel),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.conv2(x)
        return x
    

class begin_model():
    # Give as parameter the input and output channel
    def __init__(self, input_channel, output_channel):
        super(begin_model, self).__init__()
        self.first_step = layer_conv2(input_channel, output_channel)

    def forward(self, x):
        x = self.first_step(x)
        return x
    
class end_model():
    # Give as parameter the input and output channel
    def __init__(self, input_channel, output_channel):
        super(end_model, self).__init__()
        self.final_step = nn.Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=1)

    def forward(self, x):
        x = self.final_step(x)
        return x
    

# Classes necessary for 'creating' the U shape of the architecture
    
class down(nn.Module):

    def __init__(self, input_channel, output_channel):
        super(down, self).__init__()
        self.move_down = nn.Sequential(
            nn.MaxPool2d(kernel_size=2),
            layer_conv2(input_channel, output_channel)
        )

    def forward(self, x):
        x = self.move_down(x)
        return x

class up():

    def __init__(self, input_channel, output_channel):
        super(up, self).__init__()
        self.move_up = nn.Sequential(

        )

        self.move_up = layer_conv2(input_channel, output_channel)

    def forward(x):

        return x

class complete_model():
    # We give the number of classes expected and the number of channels
    def __init__(self, num_chan, num_class):
        super(complete_model, self).__init__()
        self.debut = begin_model(num_chan, output_channel=64)
        self.d1 = down(input_channel=64, output_channel=128)
        self.d2 = down(input_channel=128, output_channel=256)
        self.u1 = up(input_channel=256, output_channel=128)
        self.u2 = up(input_channel=128, output_channel=64)
        self.fin = end_model(input_channel=64, output_channel=num_class)

    def forward(self, x):
        x = self.debut(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.u1(x)
        x = self.u2(x)
        x = self.fin(x)
        return torch.sigmoid(x)
